# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [27]:
# Import Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
import json
import citipy
import scipy.stats as st

from scipy.stats import linregress
from citipy import citipy

# Import API key
from api_keys import weather_api_key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [28]:
#read the cities.csv using pd.read_csv
##documentation: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
###looks like definiing the encoding is key. use utf-8
####i thought having it defined in the WeatherPy would work, but throwing an error. Going to specify.
cities = pd.read_csv("output_csv", encoding='utf-8')
cities.head(20)


,City,Country,Latitude,Longitude,Maximum Temperature,Humidity,Clouds,Wind Speed,Date
0,Batemans Bay,AU,-35.7167,150.1833,53.20,79,100,3.83,1622656296
1,Cape Town,ZA,-33.9258,18.4232,62.11,80,20,4.00,1622656082
2,Saint-Joseph,RE,-21.3667,55.6167,75.58,79,95,14.41,1622656298
3,Rikitea,PF,-23.1203,-134.9692,71.87,60,98,17.29,1622656277
4,Butaritari,KI,3.0707,172.7902,82.65,79,100,15.59,1622656299
5,Shingū,JP,33.7333,135.9833,63.91,93,99,6.13,1622656299
6,Ushuaia,AR,-54.8000,-68.3000,35.22,93,20,5.75,1622656299
7,Chake Chake,TZ,-5.2459,39.7666,74.77,82,73,6.69,1622656300
8,Bonavista,CA,48.6499,-53.1147,69.80,66,25,9.40,1622656300
9,Dunedin,NZ,-45.8742,170.5036,45.81,88,1,1.01,1622656300


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [29]:
humidity_vac = cities["Humidity"].astype(float)
max_humidity_vac = humidity_vac.max()
locations_vac = cities[["Latitude", "Longitude"]]

In [30]:
#gmaps documentation: https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#weighted-heatmaps
figure = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations_vac, weights=humidity_vac, dissipating=False, max_intensity=max_humidity_vac, point_radius=2.5)
figure.add_layer(heat_layer)
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [31]:
#from the readme: A max temperature lower than 80 degrees but higher than 70.
##Wind speed less than 10 mph.
###Zero cloudiness.
####Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
#####Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.
narrow_city_df = cities.loc[(cities['Maximum Temperature']>70) & (cities['Maximum Temperature']<80) & (cities['Wind Speed']<10) & (cities['Clouds']==0),:]
#narrow_city_df.head()

##df worked. now use dropna. documentation:https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html
narrow_city_df = narrow_city_df.dropna(how='any')
narrow_city_df.reset_index(inplace=True)
del narrow_city_df['index']
narrow_city_df.head()

,City,Country,Latitude,Longitude,Maximum Temperature,Humidity,Clouds,Wind Speed,Date
0,Langarūd,IR,37.1970,50.1535,77.05,62,0,1.19,1622656320
1,Rørvik,NO,64.8621,11.2373,71.58,40,0,9.22,1622656455


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [32]:
#create DF and set params
#Documentation: https://developers.google.com/maps/documentation/places/web-service/search
##https://developers.google.com/maps/documentation/places/web-service/search?hl=en#PlaceSearchRequests

hotel_df = narrow_city_df
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Maximum Temperature,Humidity,Clouds,Wind Speed,Date,Hotel Name
0,Langarūd,IR,37.1970,50.1535,77.05,62,0,1.19,1622656320,
1,Rørvik,NO,64.8621,11.2373,71.58,40,0,9.22,1622656455,


In [33]:
#now look for hotels within a 5000 meter radius
for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]['Latitude']
    lon = hotel_df.loc[i]['Longitude']
    params = {
        "location" : f"{lat},{lon}",
        "radius" : 5000,
        "types" : 'lodging',
        "key" : g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    jsn = response.json()
    
    try:
        hotel_df.append(jsn['results'][0]['name']
    except:
        hotel_df.append("")

hotel_df
        

SyntaxError: invalid syntax (<ipython-input-33-416655c0b38d>, line 18)

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
